# 최근 영업일 기준 데이터 받기

In [97]:
import requests as rq 
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2023.11.17


In [98]:
# regex
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20231117


# 한국거래소의 업종분류 현황 및 개별지표 크롤링

## 업종분류 현황 크롤링

In [99]:
# OTP를 받아오는 과정

import requests as rq
from io import BytesIO
import pandas as pd 

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId' : 'STK',        # STK는 코스피
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# 헤더 부분에 레퍼러 추가 : 첫번째 URL에서 OTP를 부여받고, 이를 다시 두번째 URL에 제공하는 과정에서 레퍼러 없이 OTP를 전달하면 봇으로 인식해 데이터를 주지 않는다.
headers = {'Referer':  'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# post() 함수를 통해 해당 URL에 쿼리를 전송하면 이에 해당하는 데이터를 받으며, 이 중에 텍스트에 해당하는 내용만 불러온다.
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

E3QYxRw+guOeN5kZgZ0Hk/xqmY2x1rhf+yxdMaOW2TcRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eX8qbxyKPM+axnPoQpubhA8tBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [100]:
# OTP 제출 후 데이터 다운로드
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
# 받은 데이터의 content 부분을 ByteIO()를 이용해 바이너리 스트림 형태로 변환 후, read_csv() 함수로 데이터를 읽어 온다.
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4150,-35,-0.84,187798949850
1,006840,AK홀딩스,KOSPI,기타금융,18000,60,0.33,238456098000
2,027410,BGF,KOSPI,기타금융,3540,10,0.28,338837440140
3,282330,BGF리테일,KOSPI,유통업,133500,-6100,-4.37,2307401451000
4,138930,BNK금융지주,KOSPI,기타금융,7070,0,0.00,2277165256660


In [101]:
# 코스닥 데이터 다운로드
gen_otp_ksq = {
    'mktId': 'KSQ',      # 코스닥 코드
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2315,-35,-1.49,112362326230
1,054620,APS,KOSDAQ,금융,6860,-110,-1.58,139904356060
2,265520,AP시스템,KOSDAQ,반도체,19070,690,3.75,291416698470
3,211270,AP위성,KOSDAQ,통신장비,14550,-230,-1.56,219447523200
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,4320,160,3.85,233423748000


In [102]:
# 코스피 데이터와 코스닥 데이터 결합
# concat으로 데이터 결합, 인덱스 초기화 및 인덱스 열 삭제
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
# 종목명에 공백이 있는 경우가 있으므로 strip() 메서드를 이용해 이를 제거
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
# 기준일 열 추가
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4150,-35,-0.84,187798949850,20231117
1,006840,AK홀딩스,KOSPI,기타금융,18000,60,0.33,238456098000,20231117
2,027410,BGF,KOSPI,기타금융,3540,10,0.28,338837440140,20231117
3,282330,BGF리테일,KOSPI,유통업,133500,-6100,-4.37,2307401451000,20231117
4,138930,BNK금융지주,KOSPI,기타금융,7070,0,0.00,2277165256660,20231117


## 개별 종목 지표 크롤링

In [103]:
import requests as rq 
from io import BytesIO
import pandas as pd 

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2315,-35,-1.49,30.0,77.17,NaN,NaN,947.0,2.44,0,0.00,20231117
1,095570,AJ네트웍스,4150,-35,-0.84,201.0,20.65,612.0,6.78,8076.0,0.51,270,6.51,20231117
2,006840,AK홀딩스,18000,60,0.33,NaN,NaN,NaN,NaN,41948.0,0.43,200,1.11,20231117
3,054620,APS,6860,-110,-1.58,505.0,13.58,NaN,NaN,10864.0,0.63,0,0.00,20231117
4,265520,AP시스템,19070,690,3.75,5463.0,3.49,5685.0,3.35,17980.0,1.06,270,1.42,20231117


# 데이터 정리하기

In [104]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['제이알글로벌리츠', 'SK리츠', '씨케이에이치', '소마젠', '윙입푸드', '엑세스바이오', '프레스티지바이오파마', 'ESR켄달스퀘어리츠', 'NH올원리츠', '신한알파리츠', '맥쿼리인프라', '헝셩그룹', '오가닉티코스메틱', 'KB스타리츠', '한국패러랠', '코오롱티슈진', '모두투어리츠', '로스웰', '네오이뮨텍', '고스트스튜디오', '미래에셋글로벌리츠', '삼성FN리츠', 'GRT', 'SBI핀테크솔루션즈', '신한서부티엔디리츠', '롯데리츠', '이지스밸류리츠', '코람코더원리츠', '케이탑리츠', '한화리츠', '한국ANKOR유전', '맵스리얼티1', '컬러레이', '바다로19호', '이지스레지던스리츠', '엘브이엠씨홀딩스', '디앤디플랫폼리츠', '이리츠코크렙', '미래에셋맵스리츠', '골든센츄리', '잉글우드랩', '크리스탈신소재', '이스트아시아홀딩스', 'JTC', '글로벌에스엠', '마스턴프리미어리츠', '애머릿지', '에이리츠', '코람코라이프인프라리츠', 'NH프라임리츠']


In [105]:
# 선박펀드, 광물펀드, 해외종목 등 일반적이지 않은 종목들이라 다음 두 데이터를 합쳐 준다.
kor_ticker = pd.merge(
    krx_sector,
    krx_ind,
    on=krx_sector.columns.intersection(
        krx_ind.columns
    ).to_list(), how='outer'
)

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4150,-35,-0.84,187798949850,20231117,201.0,20.65,612.0,6.78,8076.0,0.51,270.0,6.51
1,006840,AK홀딩스,KOSPI,기타금융,18000,60,0.33,238456098000,20231117,NaN,NaN,NaN,NaN,41948.0,0.43,200.0,1.11
2,027410,BGF,KOSPI,기타금융,3540,10,0.28,338837440140,20231117,247.0,14.33,NaN,NaN,16528.0,0.21,110.0,3.11
3,282330,BGF리테일,KOSPI,유통업,133500,-6100,-4.37,2307401451000,20231117,11203.0,11.92,12512.0,10.67,55724.0,2.40,4100.0,3.07
4,138930,BNK금융지주,KOSPI,기타금융,7070,0,0.00,2277165256660,20231117,2404.0,2.94,2440.0,2.90,30468.0,0.23,625.0,8.84


In [106]:
# 마지막으로 일반적인 종목과 스펙, 우선주, 리츠, 기타 주식을 구분해주록 한다.

print(kor_ticker[kor_ticker['종목명'].str.contains('스펙|제[0-9]+호')]['종목명'].values)

['IBKS제19호스팩' 'IBKS제20호스팩' 'IBKS제21호스팩' 'IBKS제22호스팩' 'KB제25호스팩' 'KB제26호스팩'
 'KB제27호스팩' '대신밸런스제13호스팩' '대신밸런스제14호스팩' '대신밸런스제15호스팩' '대신밸런스제16호스팩'
 '비엔케이제1호스팩' '상상인제3호스팩' '상상인제4호스팩' '신한제10호스팩' '신한제11호스팩' '신한제8호스팩'
 '신한제9호스팩' '에스케이증권제10호스팩' '에스케이증권제8호스팩' '에스케이증권제9호스팩' '에이치엠씨제6호스팩'
 '유안타제10호스팩' '유안타제11호스팩' '유안타제12호스팩' '유안타제13호스팩' '유안타제14호스팩' '유안타제9호스팩'
 '케이비제21호스팩' '케이비제22호스팩' '키움제6호스팩' '키움제7호스팩' '키움제8호스팩' '하이제6호스팩' '하이제7호스팩'
 '하이제8호스팩' '한국제11호스팩' '한국제12호스팩' '한국제13호스팩' '한화플러스제2호스팩' '한화플러스제3호스팩'
 '한화플러스제4호스팩']


In [107]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['BYC우' 'CJ4우(전환)' 'CJ씨푸드1우' 'CJ우' 'CJ제일제당 우' 'DL우' 'DL이앤씨2우(전환)' 'DL이앤씨우'
 'GS우' 'JW중외제약2우B' 'JW중외제약우' 'LG생활건강우' 'LG우' 'LG전자우' 'LG화학우' 'LX하우시스우'
 'LX홀딩스1우' 'NH투자증권우' 'NPC우' 'S-Oil우' 'SK디스커버리우' 'SK우' 'SK이노베이션우' 'SK증권우'
 'SK케미칼우' '계양전기우' '금강공업우' '금호건설우' '금호석유우' '깨끗한나라우' '남선알미우' '남양유업우' '넥센우'
 '넥센타이어1우B' '노루페인트우' '노루홀딩스우' '녹십자홀딩스2우' '대교우B' '대덕1우' '대덕전자1우' '대상우'
 '대상홀딩스우' '대신증권2우B' '대신증권우' '대원전선우' '대한제당우' '대한항공우' '덕성우' '동부건설우' '동양2우B'
 '동양우' '동원시스템즈우' '두산2우B' '두산우' '두산퓨얼셀1우' '두산퓨얼셀2우B' '롯데지주우' '롯데칠성우'
 '미래에셋증권2우B' '미래에셋증권우' '부국증권우' '삼성SDI우' '삼성물산우B' '삼성전기우' '삼성전자우' '삼성화재우'
 '삼양사우' '삼양홀딩스우' '서울식품우' '성문전자우' '성신양회우' '세방우' '솔루스첨단소재1우' '솔루스첨단소재2우B'
 '신영증권우' '신풍제약우' '아모레G3우(전환)' '아모레G우' '아모레퍼시픽우' '유안타증권우' '유유제약1우'
 '유유제약2우B' '유한양행우' '유화증권우' '일양약품우' '진흥기업2우B' '진흥기업우B' '코리아써우' '코리아써키트2우B'
 '코오롱글로벌우' '코오롱모빌리티그룹우' '코오롱우' '코오롱인더우' '크라운제과우' '크라운해태홀딩스우' '태양금속우'
 '태영건설우' '티와이홀딩스우' '하이트진로2우B' '하이트진로홀딩스우' '한국금융지주우' '한양증권우' '한진칼우' '한화3우B'
 '한화갤러리아우' '한화솔루션우' '한화우' '한화투자증권우' '현대건설우' '현대차2우B' '현대차3우B' '현대차우

In [108]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

['ESR켄달스퀘어리츠' 'KB스타리츠' 'NH올원리츠' 'NH프라임리츠' 'SK리츠' '디앤디플랫폼리츠' '롯데리츠'
 '마스턴프리미어리츠' '모두투어리츠' '미래에셋글로벌리츠' '미래에셋맵스리츠' '삼성FN리츠' '신한서부티엔디리츠' '신한알파리츠'
 '에이리츠' '이지스레지던스리츠' '이지스밸류리츠' '제이알글로벌리츠' '케이탑리츠' '코람코더원리츠' '코람코라이프인프라리츠'
 '한화리츠']


In [109]:
# 해당 종목들을 구분하여 표기
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff), '기타', '보통주'))))

kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')        # 열이름 공백 제거
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})     # SQL에는 NaN이 입력되지 않으므로 None으로 변경
# kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4150,187798949850,20231117,201.0,612.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18000,238456098000,20231117,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3540,338837440140,20231117,247.0,None,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,133500,2307401451000,20231117,11203.0,12512.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7070,2277165256660,20231117,2404.0,2440.0,30468.0,625.0,보통주


In [122]:
import pymysql

con = pymysql.connect(user='root',
                      passwd='04250629',
                      host='127.0.0.1',
                      db='stock',
                      charset='utf8')

mycursor = con.cursor()
query = f""" 
INSERT INTO ticker_kr (code, name, market, close, market_cap, date, eps, forward_eps, bps, dividend, category)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
name=VALUES(name), market=VALUES(market), close=VALUES(close), market_cap=VALUES(market_cap), eps=VALUES(eps),
forward_eps=VALUES(forward_eps), bps=VALUES(bps), dividend=VALUES(dividend), category=VALUES(category);
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()